# Efficient representation of multidimensional arrays.
### Last modification (06.04.2018)


In this tutorial we provide a theoretical backgound on efficient representation of multidimensional arrays and show how these data structures are integrated into [hottbox](https://github.com/hottbox/hottbox) through **TensorCPD**, **TensorTKD** and **TensorTT** classes.

More details on **TensorCPD**, **TensorTKD** and **TensorTT** classes can be found on the [documentation page](https://hottbox.github.io/stable/api/hottbox.core.html#module-hottbox.core).

> **Note:** this tutorial assumes that you are familiar with the basics of tensor algebra and the corresponding conventional notation.  If you are new to this area, the required background is covered in our [introductory notebook](https://github.com/hottbox/hottbox-tutorials/blob/master/1_N-dimensional_arrays_and_Tensor_class.ipynb).

**Author:** Ilya Kisil - ilyakisil@gmail.com

$
\newcommand{\ten}[1]{\mathbf{\underline{#1}}}
\newcommand{\mat}[1]{\mathbf{#1}}
\newcommand{\R}{\mathbb{R}}
\newcommand{\td}[1]{\Big[ #1 \Big]}
$
### Cell with the custom latex commands:
The following commands are defined here:
- **``\ten{#1}``** - for tensor notation $\rightarrow  \ten{X}$
- **``\mat{#1}``** - for matrix notation $\rightarrow  \mat{A}$
- **``\td{#1}``** - for compact representaiton of TD (can be combined with the commands above) $\rightarrow \td{\ten{X}; \mat{A}, \mat{B}, \mat{C}}$
- **``\R``** - for subspace of real numbers ($\rightarrow  \R$)


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from hottbox.core.structures import Tensor, TensorCPD, TensorTKD, TensorTT

![storage_complexity](./images/storage_complexity.png)

# Outer product, rank-1 tensor and definitions of rank of a multi-dimensional array.

## Outer product
The central operator in tensor analysis is the outer product (sometimes refered to as the tensor product). 
Consider tensors $\ten{A} \in \R^{I_1 \times \cdots \times I_N}$ and $\ten{B}  \in \R^{J_1 \times \cdots \times J_M}$, then  their outer product yeilds a tensor of higher order then both of them:

$$
\begin{equation}
\begin{aligned}
    \ten{A} \circ \ten{B} &= \ten{C} \in \R^{I_1 \times \cdots \times I_N \times J_1 \times \cdots \times J_M} \\
    a_{i_1,\dots,i_N}b_{j_1,\dots,j_M} &= c_{i_1,\dots,i_N,j_1,\dots,j_M} 
\end{aligned}    
\end{equation}
$$

Most of the time we deal with the outer product of vectors, which significanlty simplifies the general form expressed above and establishes one the of the most fundamenatal definitions.
A tensor of order $N$ is said to be of **rank-1** if it can be represented as an outer product of $N$ vectors. The figure below illustrates an example of rank-1 tensor $\ten{X}$ and provides intuition of how operation of outer product is computed:

![outerproduct](./images/outerproduct_3.png)

There are several forms of the rank of N-dimensional arrays each of which is accosiated with a representation of a tensor in a particular form:

1. Kryskal rank $\rightarrow$ canonical polyadic form.

- Multi-linear rank $\rightarrow$ tucker form.

- TT rank $\rightarrow$ tensor train form.

> **Note:** The Kryskal rank and the rank of tensor are often time used interchangeably

Each of this representations has the correposing class: **``TensorCPD``**, **``TensorTKD``**,
**``TensorTT``**. All of them come with almost identical API except of obejct creation and,
as a result, the names for some attributes. But before, we can proceed, it is crucial to 
get acquainted with the following definitions.


## Kryskal rank

The rank of a tensor $\ten{X}$ is defined as the smallest number of rank-one tensors that generate $\ten{X}$ as their linear combination. 
The figure below illustrates a tensor $\ten{X}$ of rank $R$.

![cpd_as_rank_one](./images/cpd_as_rank_one.png)


## Multi-linear rank

The multi-linear rank of a tensor $\ten{X} \in \R^{I_1 \times \cdots \times I_N}$ is the 
$N$-tuple $(R_1, \dots, R_N)$ where each $R_n$ is the rank of the subspace spanned by mode-$n$ 
fibers, i.e. $R_n = \text{rank} \big( \mat{X}_{(n)} \big)$.

> **Note:** for a tensor of order $N$ the values $R_1, R_2, \dots , R_N$ are not necessarily the
same, whereas, for matrices (tensors of order 2) the equality $R_1 = R_2$ always holds, where 
$R_1$ and $R_2$ are the matrix column rank and row rank respectively.


## **TT rank (COMMING SOON)**

> **!!!  NEED TO FILL IN  !!!**






# Canonical polydiac form and **TensorCPD** class

![tensorcpd](./images/TensorCPD.png)

The canonical polyadic (CP) form represents a tensor as linear combination of rank-1 tensors. For a third order tensor or rank $R$ it can be expressed as follows:

$$\ten{X} = \sum_{r=1}^R \lambda_{r} \cdot \mat{a}_r \circ \mat{b}_r \circ \mat{c}_r$$

The vectors $\mat{a}_r, \mat{b}_r$ and $\mat{c}_r$ are oftentime combined into corresponding **factor matrices**:

$$
\mat{A} = \Big[ \mat{a}_1 \cdots \mat{a}_R \Big] \quad
\mat{B} = \Big[ \mat{b}_1 \cdots \mat{b}_R \Big] \quad
\mat{C} = \Big[ \mat{c}_1 \cdots \mat{c}_R \Big] \quad
$$

Thus, if we employ the mode-$n$ product, the canonical polyadic representation takes form:

$$
\ten{X} = \ten{\Lambda} \times_1 \mat{A} \times_2 \mat{B} \times_3 \mat{C} = \td{\ten{\Lambda}; \mat{A}, \mat{B}, \mat{C} }
$$
where the elements on the super-diagonal of $\ten{\Lambda}$ are occupied by the values
$\lambda_r$ and all other equal to zero.

In **``hottbox``** this form is available through the **``TensorCPD``** class.
In order to create such object, you need to pass a list of factor matrices (2d ``numpy`` arrays) and a vector of values (as 1d ``numpy`` array) for the main diagonal:

```python
tensor_cpd = TensorCPD(fmat=[A, B, C], core_values=values)
```

> **Note:** all matrices should have the same number of columns and be equal to the length of ``values``


In [2]:
tensor_rank = 4
I = 2
J = 4
K = 5

A = np.arange(I * tensor_rank).reshape(I, tensor_rank)
B = np.arange(J * tensor_rank).reshape(J, tensor_rank)
C = np.arange(K * tensor_rank).reshape(K, tensor_rank)
values = np.arange(tensor_rank)
# values = np.random.rand(tensor_rank)

tensor_cpd = TensorCPD(fmat=[A, B, C], core_values=values)

Even though we passed only the values for the super-diagonal, the core tensor of the CP 
representation still can be accessed by calling the corresponding property:

```python
tensor_cpd.core
```
This return an object of the **``Tensor``** class with the **``values``** placed on its super-diagonal.

In [3]:
print(type(tensor_cpd.core))
print('This tensor is of order {}.'.format(tensor_cpd.core.order))
print('The sizes of its modes are {} respectively.'.format(tensor_cpd.core.shape))
print('It consists of {} elemetns.'.format(tensor_cpd.core.size))
print('Its Frobenious norm = {:.2f}'.format(tensor_cpd.core.frob_norm))
tensor_cpd.core.data

<class 'hottbox.core.structures.Tensor'>
This tensor is of order 3.
The sizes of its modes are (4, 4, 4) respectively.
It consists of 64 elemetns.
Its Frobenious norm = 3.74


array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 2., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 3.]]])

In order to convert **``TensorCPD``** into the full representation, simply call: 

```python
tensor_cpd.reconstruct
```

This return an object of the **``Tensor``** class with N-dimensional array calculated as 
described above and being assinged to the **``data``** attibute.

In [4]:
tensor_full = tensor_cpd.reconstruct

print(type(tensor_full))
print('This tensor is of order {}.'.format(tensor_full.order))
print('The sizes of its modes are {} respectively.'.format(tensor_full.shape))
print('It consists of {} elemetns.'.format(tensor_full.size))
print('Its Frobenious norm = {:.2f}'.format(tensor_full.frob_norm))

tensor_full.data

<class 'hottbox.core.structures.Tensor'>
This tensor is of order 3.
The sizes of its modes are (2, 4, 5) respectively.
It consists of 40 elemetns.
Its Frobenious norm = 20607.76


array([[[   98.,   242.,   386.,   530.,   674.],
        [  242.,   610.,   978.,  1346.,  1714.],
        [  386.,   978.,  1570.,  2162.,  2754.],
        [  530.,  1346.,  2162.,  2978.,  3794.]],

       [[  242.,   610.,   978.,  1346.,  1714.],
        [  610.,  1586.,  2562.,  3538.,  4514.],
        [  978.,  2562.,  4146.,  5730.,  7314.],
        [ 1346.,  3538.,  5730.,  7922., 10114.]]])

# Tucker form and **TensorTKD** class

![tensortkd](./images/TensorTKD.png)

For a tensor $\ten{X} \in \R^{I \times J \times K}$ illustrated above, the tucker form
represents it as a dense core tensor $\ten{G}$ with multi-linear rank ($Q, R, P$) and a set of
factor matrices $\mat{A} \in \R^{I \times Q}, \mat{B} \in \R^{J \times R}$ and $\mat{C} \in
\R^{K \times P}$.

The tucker form of a tensor is closely related to the CP form and can be expressed through a 
sequence of mode-$n$ products in a similar way.

$$
\ten{X} = \ten{G} \times_1 \mat{A} \times_2 \mat{B} \times_3 \mat{C} = \td{\ten{G}; \mat{A}, \mat{B}, \mat{C} }
$$

In **``hottbox``** this form is available through the **``TensorTKD``** class.
In order to create such object, you need to pass a list of factor matrices (2d ``numpy`` arrays) and a core tensor of **``Tensor``** class:

```python
tensor_tkd = TensorTKD(fmat=[A, B, C], core=core_tensor)
```

In [5]:
I, J, K = 5, 6, 7
Q, R, P = 2, 3, 4

A = np.arange(I * Q).reshape(I, Q)
B = np.arange(J * R).reshape(J, R)
C = np.arange(K * P).reshape(K, P)

# core_array = np.random.rand(Q * R * P).reshape(Q, R, P)
core_array = np.arange(Q * R * P).reshape(Q, R, P)
core_tensor = Tensor(core_array)

tensor_tkd = TensorTKD(fmat=[A, B, C], core=core_tensor)

In order to convert **``TensorTKD``** into the full representation, simply call: 

```python
tensor_tkd.reconstruct
```

This return an object of the **``Tensor``** class with N-dimensional array calculated as 
described above and being assinged to the **``data``** attibute.

In [6]:
tensor_full = tensor_tkd.reconstruct

print(type(tensor_full))
print('This tensor is of order {}.'.format(tensor_full.order))
print('The sizes of its modes are {} respectively.'.format(tensor_full.shape))
print('It consists of {} elemetns.'.format(tensor_full.size))
print('Its Frobenious norm = {:.2f}'.format(tensor_full.frob_norm))

tensor_full.data

<class 'hottbox.core.structures.Tensor'>
This tensor is of order 3.
The sizes of its modes are (5, 6, 7) respectively.
It consists of 210 elemetns.
Its Frobenious norm = 3535670.11


array([[[    378,    1346,    2314,    3282,    4250,    5218,    6186],
        [   1368,    4856,    8344,   11832,   15320,   18808,   22296],
        [   2358,    8366,   14374,   20382,   26390,   32398,   38406],
        [   3348,   11876,   20404,   28932,   37460,   45988,   54516],
        [   4338,   15386,   26434,   37482,   48530,   59578,   70626],
        [   5328,   18896,   32464,   46032,   59600,   73168,   86736]],

       [[   1458,    5146,    8834,   12522,   16210,   19898,   23586],
        [   5112,   17944,   30776,   43608,   56440,   69272,   82104],
        [   8766,   30742,   52718,   74694,   96670,  118646,  140622],
        [  12420,   43540,   74660,  105780,  136900,  168020,  199140],
        [  16074,   56338,   96602,  136866,  177130,  217394,  257658],
        [  19728,   69136,  118544,  167952,  217360,  266768,  316176]],

       [[   2538,    8946,   15354,   21762,   28170,   34578,   40986],
        [   8856,   31032,   53208,   75384,   

# Tensor train form and **TensorTT** class

![tensortt](./images/TensorTT.png)